In [1]:
# Import libraries and set desired options
from itertools import permutations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy import sparse, stats
from scipy.linalg import svd
import umap
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     cross_validate, train_test_split)
from tqdm import tqdm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from code.cross_validation import *
from code.read_data import *
from code.feature_engineering import *
from code.autoencoder import *

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
sns.set()
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

# Read data sets

In [3]:
%%time
X1, X2, X3, Y, X1_test, X2_test, X3_test = read_data()
targets = [col for col in Y.columns if col != 'id']
train_len = len(X1)

Wall time: 910 ms


## Deal with X2 features

In [4]:
%%time
# keep categories existing in both train and test
good_A_labels = set(X2['A'].values) & set(X2_test['A'].values)
X2['A'] = X2['A'].apply(lambda x: x if x in good_A_labels else -1)
X2_test['A'] = X2_test['A'].apply(lambda x: x if x in good_A_labels else -1)
# compress class categories
#def perm(arr):
#    return set([''.join(list(str(i) for i in item)) for item in permutations(arr)])
#
#all_possible_targets = perm([0, 0, 0, 0, 0]) | \
#                       perm([0, 0, 0, 0, 1]) | \
#                       perm([0, 0, 0, 1, 1]) | \
#                       perm([0, 0, 1, 1, 1]) | \
#                       perm([0, 1, 1, 1, 1]) | \
#                       perm([1, 1, 1, 1, 1])
#tmp = pd.merge(X2, Y, how='left', on='id')
#tmp['concat'] = tmp['target_1'].astype(str) + \
#                tmp['target_2'].astype(str) + \
#                tmp['target_3'].astype(str) + \
#                tmp['target_4'].astype(str) + \
#                tmp['target_5'].astype(str)
#for i, target in enumerate(all_possible_targets):
#    can_be_compressed = set(tmp[tmp['concat'] == target]['A'].values) - set(tmp[tmp['concat'] != target]['A'].values)
#    X2['A'] = X2['A'].apply(lambda x: x if x not in can_be_compressed else -(i + 1))
#    X2_test['A'] = X2_test['A'].apply(lambda x: x if x not in can_be_compressed else -(i + 1))

Wall time: 730 ms


In [5]:
X2_all = pd.concat([X2, X2_test], ignore_index=True)
len(set(X2_all['A'].values))

53116

In [6]:
from sortedcontainers import SortedSet
X2_agg = aggregate_X2(X2, X2_test)
X2_agg['A'] = X2_agg['A'].apply(lambda x: SortedSet(x))
#X2_agg['A'].values

NameError: name 'aggregate_X2' is not defined

In [7]:
%%time
def aggregate_X2(X2, X2_test):
    X2_agg = pd.concat([X2, X2_test], ignore_index=True)
    X2_agg['A'] = X2_agg['A'].apply(lambda x: [x])
    X2_agg = X2_agg.groupby('id').agg(sum).reset_index()
    X2_agg['A'] = X2_agg['A'].apply(lambda x: set(x))
    return X2_agg

    
def get_X2_features(most_frequent_A):
    X2_all_cp = aggregate_X2(X2, X2_test)
    print(len(most_frequent_A['A'].values))
    for item in tqdm(most_frequent_A['A'].values):
        X2_all_cp[f'A_feature_{item}'] = X2_all_cp['A'].apply(lambda x: item in x)
    X2_all_cp.drop(columns=['A'], inplace=True)
    return X2_all_cp


occurences = X2_all.groupby('A')['id'].nunique().reset_index()
occurences['good'] = occurences['A'].apply(lambda x: x in good_A_labels)
occurences = occurences[occurences['good']]

Wall time: 188 ms


In [8]:
%%time
# autoencoder
X2_all_large = get_X2_features(occurences[occurences['id'] >= 5])
ids = X2_all_large['id']
X2_all_encoded = encode(X2_all_large.drop(columns=['id']), encoding_dim=32)
X2_all_encoded['id'] = ids

26508


100%|██████████| 26508/26508 [02:33<00:00, 172.46it/s]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 6043 samples, validate on 2015 samples
Epoch 1/50
 - 9s - loss: 0.5480 - val_loss: 0.2008
Epoch 2/50
 - 5s - loss: 0.1012 - val_loss: 0.0610
Epoch 3/50
 - 5s - loss: 0.0543 - val_loss: 0.0512
Epoch 4/50
 - 5s - loss: 0.0496 - val_loss: 0.0492
Epoch 5/50
 - 5s - loss: 0.0485 - val_loss: 0.0485
Epoch 6/50
 - 5s - loss: 0.0480 - val_loss: 0.0481
Epoch 7/50
 - 5s - loss: 0.0476 - val_loss: 0.0477
Epoch 8/50
 - 5s - loss: 0.0471 - val_loss: 0.0472
Epoch 9/50
 - 5s - loss: 0.0465 - val_loss: 0.0463
Epoch 10/50
 - 5s - loss: 0.0452 - val_loss: 0.0450
Epoch 11/50
 - 5s - loss: 0.0435 - val_loss: 0.0436
Epoch 12/50
 - 5s - loss: 0.0419 - val_loss: 0.0424
Epoch 13/50
 - 5s - loss: 0.0408 - val_loss: 0.0415
Epoch 14/50
 - 5s - loss: 0.0398 - val_loss: 0.0407
Epoch 15/50
 - 5s - loss: 0.0389 - val_loss: 0.0399
Epoch 16/50
 - 5s - loss: 0.0379 - val_loss: 0.0392
Epoch 17/

In [13]:
%%time
# X1
X1_all = pd.concat([X1, X1_test], ignore_index=True)
bool_columns = [col for col in X1_all.columns if X1_all[col].nunique() == 2]
not_bool_columns = list(set(X1_all.columns) - set(bool_columns) - set(['id']))
# X1 categorical
X1_categorical_embedding = umap.UMAP(n_components=3, metric='dice').fit_transform(X1_all[bool_columns])
X1_categorical_embedding_df = pd.DataFrame(X1_categorical_embedding, columns=[f'X1_all_embedding_{i+1}' for i in range(3)])
X1_categorical_embedding_df['id'] = X1_all['id']
# X1 other
X1_other_embedding = umap.UMAP().fit_transform(X1_all[not_bool_columns])
X1_other_embedding_df = pd.DataFrame(X1_other_embedding, columns=['X1_all_embedding_5', 'X1_all_embedding_6'])
X1_other_embedding_df['id'] = X1_all['id']
# X2
X2_all_cp = get_X2_features(occurences[occurences['id'] >= 5])
X2_all_embedding = umap.UMAP(n_components=32, metric='dice').fit_transform(X2_all_cp.drop(columns=['id']))
X2_all_embedding_df = pd.DataFrame(X2_all_embedding, columns=[f'X2_all_embedding_{i+1}' for i in range(32)])
X2_all_embedding_df['id'] = X2_all_cp['id']
# unite
embeddings = [X1_categorical_embedding_df, X1_other_embedding_df, X2_all_embedding_df]

26508


100%|██████████| 26508/26508 [02:32<00:00, 174.19it/s]


Wall time: 1h 42min 39s


In [14]:
import pickle
with open('data/embeddings_5_n.pickle','wb') as f:
    pickle.dump(embeddings, f)

In [15]:
%%time
X_train = agg_and_merge(X1, X2, X3, embeddings)
X_test = agg_and_merge(X1_test, X2_test, X3_test, embeddings)
X_train, X_test = eng(X_train, X_test)
X_train, X_test = add_emedding_features(X_train, X_test, random_state=42)
X_train_norm, X_test_norm = normalize(X_train, X_test)

umap...
Wall time: 54 s


In [16]:
%%time
X_train_norm = pd.merge(X_train_norm, X2_all_encoded, how='left', on='id')
X_test_norm = pd.merge(X_test_norm, X2_all_encoded, how='left', on='id')
X_train = pd.merge(X_train, X2_all_encoded, how='left', on='id')
X_test = pd.merge(X_test, X2_all_encoded, how='left', on='id')

X_train_norm.fillna(-999, inplace=True)
X_test_norm.fillna(-999, inplace=True)
X_train.fillna(-999, inplace=True)
X_test.fillna(-999, inplace=True)
assert len(X_train) == 4000

Wall time: 304 ms


In [17]:
%%time
X_train_norm_ext = pd.merge(X_train_norm, X2_all_large, how='left', on='id')
X_test_norm_ext = pd.merge(X_test_norm, X2_all_large, how='left', on='id')
X_train_ext = pd.merge(X_train, X2_all_large, how='left', on='id')
X_test_ext = pd.merge(X_test, X2_all_large, how='left', on='id')

X_train_norm_ext.fillna(-999, inplace=True)
X_test_norm_ext.fillna(-999, inplace=True)
X_train_ext.fillna(-999, inplace=True)
X_test_ext.fillna(-999, inplace=True)
assert len(X_train_norm_ext) == 4000

Wall time: 2.53 s


In [18]:
%%time
et_multi = ExtraTreesClassifier(n_estimators=1000,
                                max_depth=7,
                                class_weight='balanced',
                                random_state=42,
                                n_jobs=4)
CV_multilabel(et_multi, X_train_ext, Y.drop(columns=['id']))

target_1: 0.549, [0.5437299582848549, 0.5449122490683976, 0.5139370242009539, 0.5618934130749227, 0.5805197523849397]
target_2: 0.618, [0.6043129527351447, 0.6121239009437767, 0.6010181818181818, 0.6303199404761904, 0.6444586541862806]
target_3: 0.607, [0.5875517179348804, 0.6117143371109912, 0.6034869025735294, 0.6073494960133407, 0.6261058134191176]
target_4: 0.604, [0.6094420929947245, 0.6058032207384131, 0.6353583532936198, 0.5837193666836595, 0.5872379072627936]
target_5: 0.582, [0.573900148294612, 0.54798508884112, 0.5854133333333333, 0.5884937061755513, 0.6120513727517619]
0.5921135553838036
Wall time: 52.4 s


In [19]:
%%time
rf_multi = RandomForestClassifier(n_estimators=1000,
                                  max_depth=10,
                                  class_weight='balanced',
                                  random_state=42,
                                  n_jobs=4)
CV_multilabel(rf_multi, X_train_ext, Y.drop(columns=['id']))

target_1: 0.549, [0.5564431453786998, 0.5474290780141844, 0.5089161408809503, 0.5653109097586019, 0.5653279105236402]
target_2: 0.624, [0.6122783158946462, 0.6157739775752199, 0.6111930735930736, 0.6349330357142857, 0.6435335092922642]
target_3: 0.603, [0.5777370030581039, 0.6345673682316963, 0.5924359489889706, 0.5767254707363674, 0.6332074333639707]
target_4: 0.601, [0.6002944423997055, 0.5990699697003703, 0.6282209690000069, 0.5955478718295395, 0.5838768271403644]
target_5: 0.593, [0.5831837972900683, 0.585718755952219, 0.5932, 0.5922319844555208, 0.6093983837174499]
0.5938622128995966
Wall time: 1min 8s


In [20]:
%%time
step = 500
intervals = [(i, i + step) for i in range(0, 4000, step)]
splits = [[item for j, item in enumerate(intervals) if i != j] for i in range(len(intervals))]
out_dims = [[item for j, item in enumerate(intervals) if i == j] for i in range(len(intervals))]
X_train_splits = [pd.concat([X_train_ext[lb:ub] for (lb, ub) in split]) for split in splits]
y_train_splits = [pd.concat([Y[lb:ub] for (lb, ub) in split]) for split in splits]

Wall time: 1.48 s


In [21]:
%%time
metafeatures = pd.concat([X_train[['id']], X_test[['id']]], ignore_index=True)
for model_name, model in tqdm(zip(['rf_multi', 'et_multi'], [rf_multi, et_multi])):
    X_train_predicts = []
    X_test_predicts = []
    for X_train_split, y_train_split, out_dim in zip(X_train_splits, y_train_splits, out_dims):
        model.fit(X_train_split, y_train_split.drop(columns=['id']))
        X_train_predicts.append(model.predict_proba(X_train_ext[out_dim[0][0]:out_dim[0][1]]))
        X_test_predicts.append(model.predict_proba(X_test_ext))
    
    # X train features
    X_train_features = {target: [] for target in targets}
    for chunk in X_train_predicts:
        for target, probas in zip(targets, chunk):
            X_train_features[target].append(probas[:, 1])
    for target in targets:
        X_train_features[target] = np.concatenate(X_train_features[target])
    # X test features
    X_test_features = {target: [] for target in targets}
    for chunk in X_test_predicts:
        for target, probas in zip(targets, chunk):
            X_test_features[target].append(probas[:, 1])
    for target in targets:
        X_test_features[target] = sum(X_test_features[target]) / len(X_test_features[target])
    # metafeatures
    for target in targets:
        metafeatures[f'meta_{model_name}_{target}'] = np.concatenate((X_train_features[target], X_test_features[target]))

2it [04:53, 152.44s/it]


Wall time: 4min 53s


In [22]:
%%time
X_train_norm = pd.merge(X_train_norm, metafeatures, on='id')
X_test_norm = pd.merge(X_test_norm, metafeatures, on='id')
X_train = pd.merge(X_train, metafeatures, on='id')
X_test = pd.merge(X_test, metafeatures, on='id')

Wall time: 169 ms


# Simple models

In [23]:
merged = pd.merge(X_train, Y, on='id')
logreg_merged = pd.merge(X_train_norm, Y, on='id')
assert len(merged) == len(logreg_merged)
final_models = {}
final_models_roc_auc = {}

## logregs

In [24]:
%%time
logreg = LogisticRegression(C=0.05, class_weight='balanced', random_state=42, n_jobs=4)
roc_aucs = []
for target in targets:
    roc_aucs.append(CV_metrics(logreg, X_train_norm, Y[target].values))
print(np.mean(roc_aucs))

roc_auc  avg:  0.597   ['0.581', '0.566', '0.598', '0.608', '0.634']
roc_auc  avg:  0.624   ['0.618', '0.636', '0.643', '0.607', '0.615']
roc_auc  avg:  0.619   ['0.597', '0.614', '0.641', '0.646', '0.595']
roc_auc  avg:  0.613   ['0.640', '0.626', '0.618', '0.586', '0.593']
roc_auc  avg:  0.618   ['0.635', '0.613', '0.638', '0.619', '0.583']
0.6139272302796444
Wall time: 9.12 s


## RF

In [25]:
%%time
rf = RandomForestClassifier(n_estimators=1000,
                            max_depth=6,
                            class_weight='balanced',
                            random_state=42,
                            n_jobs=4)
roc_aucs = []
for target in targets:
    roc_aucs.append(CV_metrics(rf, X_train, Y[target].values))
print(np.mean(roc_aucs))

roc_auc  avg:  0.596   ['0.553', '0.589', '0.606', '0.595', '0.640']
roc_auc  avg:  0.622   ['0.619', '0.638', '0.635', '0.607', '0.610']
roc_auc  avg:  0.613   ['0.603', '0.614', '0.622', '0.613', '0.612']
roc_auc  avg:  0.616   ['0.630', '0.617', '0.636', '0.591', '0.608']
roc_auc  avg:  0.606   ['0.618', '0.621', '0.613', '0.621', '0.557']
0.610700171713918
Wall time: 1min 10s


## ET

In [26]:
%%time
et = ExtraTreesClassifier(n_estimators=1000,
                          max_depth=9,
                          class_weight='balanced',
                          random_state=42,
                          n_jobs=4)
roc_aucs = []
for target in targets:
    roc_aucs.append(CV_metrics(et, X_train, Y[target].values))
print(np.mean(roc_aucs))

roc_auc  avg:  0.576   ['0.520', '0.569', '0.586', '0.579', '0.628']
roc_auc  avg:  0.622   ['0.619', '0.642', '0.634', '0.609', '0.605']
roc_auc  avg:  0.615   ['0.601', '0.611', '0.623', '0.625', '0.613']
roc_auc  avg:  0.616   ['0.639', '0.617', '0.626', '0.598', '0.599']
roc_auc  avg:  0.593   ['0.605', '0.604', '0.597', '0.612', '0.548']
0.6043487829418638
Wall time: 49.6 s


## GBM

In [27]:
%%time
gbm = LGBMClassifier(num_leaves=3,
                     learning_rate=0.05,
                     reg_lambda=75.0,
                     random_state=42,
                     class_weight='balanced')
roc_aucs = []
for target in targets:
    roc_aucs.append(CV_metrics(gbm, X_train, Y[target].values))
print(np.mean(roc_aucs))

roc_auc  avg:  0.616   ['0.593', '0.603', '0.618', '0.617', '0.649']
roc_auc  avg:  0.621   ['0.626', '0.635', '0.629', '0.608', '0.605']
roc_auc  avg:  0.629   ['0.597', '0.620', '0.654', '0.643', '0.629']
roc_auc  avg:  0.619   ['0.632', '0.633', '0.637', '0.591', '0.600']
roc_auc  avg:  0.604   ['0.626', '0.601', '0.601', '0.634', '0.557']
0.6175374189440908
Wall time: 13.9 s


## Model

In [28]:
%%time
probas = []
X_train = merged.drop(columns=targets)
X_train_logreg = logreg_merged.drop(columns=targets)
for target in targets:
    print(target)
    y_train = merged[target].values
    # models
    # lightgbm
    y_proba = gbm.fit(X_train, y_train).predict_proba(X_test)[:, 1]
    # random forest
    y_proba += rf.fit(X_train, y_train).predict_proba(X_test)[:, 1]
    # extra tree
    y_proba += et.fit(X_train, y_train).predict_proba(X_test)[:, 1]
    # logreg
    y_proba += logreg.fit(X_train_norm, y_train).predict_proba(X_test_norm)[:, 1]
    y_proba /= 4.0
    final_proba = y_proba
    probas.append(final_proba)

target_1
target_2
target_3
target_4
target_5
Wall time: 35.6 s


In [29]:
tmp = pd.DataFrame(probas).T
baseline = pd.DataFrame(tmp.values, columns=['1', '2', '3', '4', '5'])
baseline['id'] = X_test['id']
baseline[['id', '1', '2', '3', '4', '5']].to_csv('baseline.csv', index=False)